<a href="https://colab.research.google.com/github/Bennyhinn18/NM-project-AI-in-Aerospace-Engineering/blob/main/aerospace-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('sensor-reading.csv')

# Display the first few rows of the dataset
print(data.head())

# Explore dataset information
print(data.info())


FileNotFoundError: [Errno 2] No such file or directory: 'sensor-reading.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')